# Work in progress (WIP)

# 4. Deutsch's algorithm

In [ ]:
# Specific install for Kaggle Notebooks environment
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

# Kaleido is used to convert Plotly to static png
!pip install -Uqq kaleido
!pip install -Uqq skq

# Work in progress (WIP)

- One of the simplest algorithms exhibiting quantum advantage.

- Solves the parity problem in one pass, as opposed to classical algorithms that require at least two passes. The outcome is either constant or balanced.
- One of the query algorithms along with **Deutsch-Josza**, **Grover** and **Shor** algorithms.
- Phase kickback.


$$ U_f \vert x \rangle \vert y \rangle = \vert x \rangle \vert y \oplus f(x) \rangle $$

- $0$ if $f$ is constant. $1$ if $f$ is balanced.

- Balanced means $f(0) \neq f(1)$. Constant means $f(0) = f(1)$.

- No entanglement required.

- Gates are Hadamard and a custom Unitary function (black-box) operator.

# Work in progress (WIP)

## Deutsch

## Deutsch-Josza

- Very similar to Deutsch's algorithm.
- Extends Deutsch's algorithm to $n \geq 1$ bits.

- $U_f$ has a so-called promise. This means that we have to make sure that the function $f$ is either constant or balanced, else the algorithm will not work and will give unpredictable results. 

- Balanced function: $f(x_1, \ldots, x_n) = 1$ for exactly $2^{n-1}$ inputs and $f(x_1, \ldots, x_n) = 0$ for the remaining $2^{n-1}$ inputs.
- Constant function: $f(x_1, \ldots, x_n) = 0$ for all inputs or $f(x_1, \ldots, x_n) = 1$ for all inputs.

- Deterministic classical algorithm requires $2^{n-1} + 1$ queries to determine if the function is constant or balanced. 

- A probabilistic classical algorithm requires a few queries.

1. Choose $k$ input strings $x_1, \ldots, x_k$ at random.
2. If $f(x_i) = 1$ for all $i = 1, \ldots, k$, then output constant, else balanced. If $f$ is constant algorithm is 100% correct. if $f$ is balanced, the algorithm is correct with probability $1 - 2^{-k+1}$.

- Deutsch-Josza algorithm requires only $1$ query.

- The same Deutsch-Josza algorithm can also solve the **Bernstein-Vazirani** problem. A recursive version of Bernstein-Vazirani was one of the first algorithms that gain a superpolynomial advantage over classical algorithms.

# TODO Show both Deutsch-Josza and probabilistic classical algorithm.



# Work In Progress (WIP)